In [1]:
import numpy as np
import pufferlib, pufferlib.vector
from pufferlib.environments import classic_control

In [2]:
num_envs = 12

In [3]:
vecenv = pufferlib.vector.make(
    classic_control.env_creator("CartPole-v1"),
    num_envs=num_envs,
    backend=pufferlib.vector.Multiprocessing,
)

Process Process-11:
Process Process-3:
Process Process-1:
Process Process-7:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-5:
Process Process-4:
Process Process-12:
Process Process-10:
Process Process-6:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314

In [4]:
env_rewards = [list() for _ in range(num_envs)]
env_rewards

returns = []

from itertools import count

dones = np.array([False] * num_envs)
truncateds = np.array([False] * num_envs)
dones, truncateds

obs, _ = vecenv.reset()
episodes = 0
for t in count():
    obs, rewards, dones, truncateds, _ = vecenv.step(vecenv.action_space.sample())
    for i, reward in enumerate(rewards):
        env_rewards[i].append(reward)
    for i in np.where(dones | truncateds)[0]:
        returns.append(sum(env_rewards[i]))
        env_rewards[i] = []
        episodes += 1
    if episodes >= 1_000:
        break

np.mean([np.sum(rets) for rets in returns]), t

(22.278, 1949)

In [5]:
import torch
from torch import nn

In [6]:
class PPONetwork(nn.Module):
    def __init__(self, n_input, n_hiddens, n_actions):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(n_input, n_hiddens),
            nn.ReLU(),
        )

        self.policy_head = nn.Linear(n_hiddens, n_actions)
        self.value_head = nn.Linear(n_hiddens, 1)

    def forward(self, x):
        device = next(self.mlp.parameters()).device
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(np.array(x), dtype=torch.float32, device=device)
        x = x.to(device=device)

        x = self.mlp(x)
        logits = self.policy_head(x)
        value = self.value_head(x)
        
        return logits, value

In [7]:
import random

In [8]:
def select_actions(logits):
    dist = torch.distributions.Categorical(logits=logits) # shape [batch_size, n_actions]
    
    actions = dist.sample() # Shape: [n_actions]
    log_probs = dist.log_prob(actions)

    return actions, log_probs

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
from torch import optim

from itertools import count
from tqdm.auto import tqdm

import wandb

In [11]:
def train(
    max_batches = 8_000,
    batch_size = 2048,
    minibatch_size = 64,
    num_epochs = 10,
    learning_rate = 1e-4,
    gamma = 0.99,
    lmbd = 0.95,
    value_loss_coef = 0.5,
    clip_param = 0.2,
    leave_bar = True,
):  
    assert batch_size % minibatch_size == 0
    num_minibatches = int(batch_size / minibatch_size)
    net = PPONetwork(n_input, n_hiddens, n_actions).to(device)
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    if wandb.run is not None:
        wandb.watch(net, log="all")
        
    obs, _ = vecenv.reset()
    try:
        for batch in tqdm(range(max_batches), leave=leave_bar, desc="Batches"):
            batch_states = torch.zeros((batch_size + 1, num_envs, n_input), device=device)
            batch_values = torch.zeros((batch_size + 1, num_envs), device=device)
            batch_actions = torch.zeros((batch_size, num_envs), device=device)
            batch_log_probs = torch.zeros((batch_size, num_envs), device=device)
            batch_rewards = torch.zeros((batch_size, num_envs), device=device)
            batch_dones = torch.zeros((batch_size, num_envs), dtype=torch.bool, device=device)
            batch_advantages = torch.zeros((batch_size, num_envs), device=device)
            
            # Collect one rollout for each env
            for t in range(batch_size):
                with torch.no_grad():
                    logits, values = net(obs)
                    actions, log_probs = select_actions(logits)
        
                next_obs, rewards, dones, truncateds, infos = vecenv.step(actions.cpu())

                for i, info in enumerate(infos):
                    wandb.log(data={"return": info["episode_length"]})
                    
                batch_states[t] = torch.from_numpy(obs.copy())
                batch_values[t] = values.squeeze()
                batch_actions[t] = actions.squeeze()
                batch_log_probs[t] = log_probs
                batch_rewards[t] = torch.from_numpy(rewards.copy())
                batch_dones[t] = torch.from_numpy(dones.copy())

                obs = next_obs.copy()

            # Append the final states and values for computing the final losses
            with torch.no_grad():
                _, final_values = net(obs)
                batch_states[batch_size] = torch.from_numpy(obs.copy())
                batch_values[batch_size] = final_values.squeeze()
            
            # Calculate advantages
            advantages = torch.zeros(final_values.shape[0], device=device)
            for t in reversed(range(batch_size)):
                next_values = batch_values[t+1] * ~batch_dones[t]
                td_error = batch_rewards[t] + gamma * next_values - batch_values[t]
                advantages = td_error + gamma * lmbd * (advantages * ~batch_dones[t])
                adv_mean, adv_std = advantages.mean(), advantages.std()
                batch_advantages[t] = (advantages - adv_mean) / (1.0 if adv_std == 0.0 else adv_std)

            # Perform updates
            for epoch in range(num_epochs):
                indices = torch.randperm(batch_size)
                for minibatch in range(num_minibatches):
                    start = minibatch * minibatch_size
                    end = start + minibatch_size
                    
                    for t in indices[start:end]:
                        logits, values = net(batch_states[t])
                        dist = torch.distributions.Categorical(logits=logits)
                        log_probs = dist.log_prob(batch_actions[t])
                        ratio = (log_probs - batch_log_probs[t]).exp()
                        
                        next_values = batch_values[t+1] * ~batch_dones[t]
                        td_target = batch_rewards[t] + gamma * next_values
                        value_loss = (value_loss_coef * (td_target - values)**2).mean()
                        
                        surr1 = ratio * batch_advantages[t]
                        surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * batch_advantages[t]
                        policy_loss = -torch.min(surr1, surr2).mean()
    
                        loss = policy_loss + value_loss
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

    except KeyboardInterrupt:
        print("Training halted manually.")
        
    # Finalize the run
    model_path = "./models/PPO_scratch_CartPole_latest.pt"
    torch.save(net.state_dict(), model_path)
    wandb.log_model(model_path, "latest")
    wandb.unwatch()
    net.eval()
    
    return net

In [12]:
n_input = vecenv.single_observation_space.shape[0]
n_actions = vecenv.single_action_space.n
n_hiddens = 16

max_batches = 2_400
batch_size = 1024
minibatch_size = 128
num_epochs = 10
learning_rate = 1e-4
gamma = 0.99
lmbd = 0.95
value_loss_coef = 0.5
clip_param = 0.2
leave_bar = True

project = "PPO-scratch-CartPole"
config = {
    "num_envs": num_envs,
    "n_hiddens": n_hiddens,
    "batch_size": batch_size,
    "minibatch_size": minibatch_size,
    "num_epochs": num_epochs,
    "learning_rate": learning_rate,
    "gamma": gamma,
    "lambda": lmbd,
    "value_loss_coef": value_loss_coef,
    "clip_param": clip_param,
}

wandb.init(
    project=project,
    config=config,
)
    
net = train(
    max_batches=max_batches,
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    gamma=gamma,
    lmbd=lmbd,
    value_loss_coef=value_loss_coef,
    clip_param=clip_param,
    leave_bar=leave_bar,
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fitti. Use `wandb login --relogin` to force relogin


Batches:   0%|          | 0/2400 [00:00<?, ?it/s]

Training halted manually.


In [13]:
import gymnasium as gym
import numpy as np
from datetime import datetime
from pathlib import Path

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
video_folder = f"./videos/PPO_scratch_CartPole_{timestamp}"

env = gym.make("CartPole-v1", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, video_folder)

epsilon = 0.0

for _ in range(9):
    ob, _ = env.reset()
    ob = np.expand_dims(ob, 0)
    ret = 0
    done, truncated = False, False
    while not (done or truncated):
        logits, _ = net(ob)
        actions, *_ = select_actions(logits)
        ob, reward, done, truncated, _ = env.step(actions[0].cpu().numpy())
        ob = np.expand_dims(ob, 0)
        ret += reward
    
    print(ret)

if wandb.run is not None:
    latest_video = max(
        Path(video_folder).glob("*.mp4"),
        key=lambda x: x.stat().st_mtime
    )
    wandb.log({
        "video": wandb.Video(str(latest_video))
    })
    wandb.finish()
    
env.close()

MoviePy - Building video /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-0.mp4.
MoviePy - Writing video /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-0.mp4



MoviePy - Done !
MoviePy - video ready /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-0.mp4
500.0
MoviePy - Building video /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-1.mp4.
MoviePy - Writing video /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-1.mp4



MoviePy - Done !
MoviePy - video ready /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-1.mp4
500.0
500.0
500.0
500.0
500.0
500.0
500.0
MoviePy - Building video /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-8.mp4.
MoviePy - Writing video /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-8.mp4



MoviePy - Done !
MoviePy - video ready /home/fitti/projects/puffer/videos/PPO_scratch_CartPole_20250221_153133/rl-video-episode-8.mp4
500.0


return,▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▂▂▂▃▃▂▄▂▄▄▃▄▆█▃█▆▇█▆█
return,500
